In [2]:
# Data retrieval

In [1]:
from epigenomic_dataset import load_epigenomes
from ucsc_genomes_downloader import Genome
from keras_bed_sequence import BedSequence
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import entropy
from minepy import MINE
from boruta import BorutaPy
from prince import MFA


import pandas as pd
import numpy as np

from typing import Dict, List, Tuple

In [2]:
def download_data(cell_line: str, dataset: str = "fantom", window_size: int = 200) -> Dict[str, pd.DataFrame]:
    regions = ["promoters", "enhancers"]
    epigenomes = {}
    labels = {}
    for region in regions:
        epigenome, label = load_epigenomes(
            cell_line=cell_line,
            dataset=dataset,
            regions=region,
            window_size=window_size
        )
        epigenomes.update({region: epigenome})
        labels.update({region: label})
    return epigenomes, labels

In [3]:
def to_bed(data: pd.DataFrame)-> pd.DataFrame:
    """Return bed coordinates from given dataset."""
    return data.reset_index()[data.index.names]

In [4]:
def get_genome(assembly: str = "hg19") -> Genome:
    return Genome(assembly)

In [5]:
def one_hot_encode(data: pd.DataFrame, genome: Genome, nucleotides : str = "actg", window_size: int = 200)-> np.ndarray:
    """Set to one only the nucletoide, zero the others"""
    return np.array(BedSequence(
        genome,
        bed=to_bed(data),
        nucleotides=nucleotides,
        batch_size=1
    )).reshape(-1, window_size * len(nucleotides)).astype(int)

In [6]:
def to_dataframe(data: np.ndarray, window_size : int = 200, nucleotides : str = "actg")-> pd.DataFrame:
    return pd.DataFrame(
        data,
        columns = [
            f"{i}{nucleotide}"
            for i in range(window_size)
            for nucleotide in nucleotides
        ]
    )

In [7]:
def get_sequences(epigenomes: np.ndarray, genome: Genome, window_size: int = 200) -> Dict[str, pd.DataFrame]:
    {
    region: to_dataframe(
        one_hot_encode(data, genome, window_size),
        window_size
    )
    for region, data in epigenomes.items()
}

In [8]:
info = {}

In [9]:
def data_retrieval(cell_line: str = 'K562') -> Dict[str, pd.DataFrame]:
    global info
    print('Downloading data...')
    epigenomes, labels = info.get('data') or download_data(cell_line)
    info['data'] = (epigenomes, labels)
    print('Downloading genome...')
    genome = info.get('genome') or get_genome()
    info['genome'] = genome
    print('Getting dataframe...')
    sequences = info.get('sequences') or get_sequences(epigenomes, genome)
    info['sequences'] = sequences
    print('Finished!')
    return epigenomes, labels, sequences

In [10]:
data_retrieval()

KeyboardInterrupt: 

In [ ]:
# Data elaboration

In [ ]:
def overfitting_risk(epigenomes: Dict[str, pd.DataFrame], threshold: int = 1) -> bool:
    valid = False
    for region, data in epigenomes.items():
        rate = data[0] / data[1]
        print(f'Rate for {region} is {rate}')
        valid = valid or (rate < threshold)
    return valid

In [ ]:
def nan_check(epigenomes: Dict[str, pd.DataFrame], threshold: int = 10) -> None:
    for region, x in epigenomes.items():
        print("\n".join((
            f"Nan values report for {region} data:",
            f"In the document there are {x.isna().values.sum()} NaN values out of {x.values.size} values.",
            f"The sample (row) with most values has {x.isna().sum(axis=0).max()} NaN values out of {x.shape[1]} values.",
            f"The feature (column) with most values has {x.isna().sum().max()} NaN values out of {x.shape[0]} values."
        )))
        print("="*80)

In [ ]:
def fit_constant(data: pd.DataFrame, value: int) -> pd.DataFrame:
    return df.fillna(value)


def fit_media(data: pd.DataFrame) -> pd.DataFrame:
    return fit_constant(data, data.mean())


def fit_median(data: pd.DataFrame) -> pd.DataFrame:
    return fit_constant(data, data.median())


def fit_mode(data: pd.DataFrame) -> pd.DataFrame:
    return fit_constant(data, data.mode())


def fit_neighbours(data: pd.DataFrame, neighbours: int = 5) -> pd.DataFrame:
    return pd.DataFrame(KNNImputer(n_neighbours=neighbours).fit_transform(data.values), 
                        columns=data.columns,
                        index=data.index
                       )


def fit_missing(epigenomes: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    for region, data in epigenomes.items():
        epigenomes[region] = fit_neighbours(data)
    return epigenomes

In [ ]:
def check_class_balance(labels: Dict[str, pd.DataFrame]) -> None:
    fig, axes = plt.subplots(ncols=2, figsize=(10, 5))

    for axis, (region, y) in zip(axes.ravel(), labels.items()):
        y.hist(ax=axis, bins=3)
        axis.set_title(f"Classes count in {region}")
    fig.show()

In [ ]:
def drop_constant_features(epigenomes: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
    def drop(df:pd.Dataframe) -> pd.DataFrame:
        return df.loc[:, (df != df.iloc[0]).any()]
    for region, data in epigenomes.items():
        result = drop_constant_features(data)
        if data.shape[1] != result.shape[1]:
            print(f"Features in {region} were constant and had to be dropped!")
            epigenomes[region] = result
        else:
            print(f"No constant features were found in {region}!")
    return epigenomes

In [ ]:
def apply_z_scoring(epigenomes: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
    def robust_zscoring(df: pd.DataFrame) -> pd.DataFrame:
        return pd.DataFrame(
            RobustScaler().fit_transform(df.values),
            columns=df.columns,
            index=df.index
        )
    return {region: robust_zscoring(data) for region, data in epigenomes.items()}

In [ ]:
def drop_uncorrelated(epigenomes: Dict[str, pd.DataFrame], p_value_threshold: float = 0.01, correlation_threshold: float = 0.05) -> Dict[str, pd.DataFrame]:
    uncorrelated = {region: set() for region in epigenomes}
    
    def pearson():
        for region, data in epigenomes.items():
            for column in tqdm(data.columns, desc=f"Running Pearson test for {region}", dynamic_ncols=True, leave=False):
                correlation, p_value = pearsonr(data[column].values.ravel(), labels[region].values.ravel())
                if p_value > p_value_threshold:
                    print(region, column, correlation)
                    uncorrelated[region].add(column)
    
    def spearman():
        for region, data in epigenomes.items():
            for column in tqdm(data.columns, desc=f"Running Spearman test for {region}", dynamic_ncols=True, leave=False):
                correlation, p_value = spearmanr(data[column].values.ravel(), labels[region].values.ravel())
                if p_value > p_value_threshold:
                    print(region, column, correlation)
                uncorrelated[region].add(column)
    
    def mine():
        for column in tqdm(uncorrelated[region], desc=f"Running MINE test for {region}", dynamic_ncols=True, leave=False):
            mine = MINE()
            mine.compute_score(x[column].values.ravel(), labels[region].values.ravel())
            score = mine.mic()
            if score < correlation_threshold:
                print(region, column, score)
            else:
                uncorrelated[region].remove(column)
                
    def drop():
        for region, data in epigenomes.items():
            epigenomes[region] = data.drop(columns=[col for col in uncorrelated[region] if col in x.columns])
        return epigenomes
                
    pearson()
    spearman()
    mine()
    return drop()

In [ ]:
def drop_too_correlated(epigenomes: Dict[str, pd.DataFrame], p_value_threshold: float = 0.01, correlation_threshold: float = 0.95) -> Dict[str, pd.DataFrame]:
    extremely_correlated = {region: set() for region in epigenomes}
    scores = {region: [] for region in epigenomes}
    
    def pearson():
        for region, x in epigenomes.items():
            for i, column in tqdm(
                    enumerate(x.columns),
                    total=len(x.columns), 
                    desc=f"Running Pearson test for {region}", 
                    dynamic_ncols=True, 
                    leave=False):
                for feature in x.columns[i+1:]:
                    correlation, p_value = pearsonr(x[column].values.ravel(), x[feature].values.ravel())
                    correlation = np.abs(correlation)
                    scores[region].append((correlation, column, feature))
                    if p_value < p_value_threshold and correlation > correlation_threshold:
                        print(region, column, feature, correlation)
                        if entropy(x[column]) > entropy(x[feature]):
                            extremely_correlated[region].add(feature)
                        else:
                            extremely_correlated[region].add(column)
                        
    return {region: sorted(score, key=lambda x: np.abs(x[0]), reverse=True) for region, score in scores.items()}

In [ ]:
def show(epigenomes: Dict[str, pd.DataFrame], scores: Dict[str, List[Tuple]]):
    def get_data(region:str, how_many: int, from_start: bool = True) -> List[str]:
        data = scores[region][:how_many] if from_start else scores[region][-how_many:]
        data = list(zip(data))[1:]
        columns = list(set([column for row in data for column in row]))
        return columns
    
    def plot(how_many: int, correlated: bool) -> None:
        for region, data in epigenomes.items():
            print(f"Most {'correlated' if correlated else 'uncorrelated'} features from {region} epigenomes")
            sns.pairplot(pd.concat([
                x[get_data(region, how_many, correlated)],
                labels[region],
            ], axis=1), hue=labels[region].columns[0])
            plt.show()
    
    def correlated(how_many: int):
        plot(how_many, True)
        
    def uncorrelated(how_many: int):
        plot(how_many, False)
        
    def get_top_most_different(dist, n: int):
        return np.argsort(-np.mean(dist, axis=1).flatten())[:n]
    
    def get_top_most_different_tuples(dist, n: int):
        return list(zip(*np.unravel_index(np.argsort(-dist.ravel()), dist.shape)))[:n]
        
    def most_different_features(top_number: int) -> None:
        for region, x in epigenomes.items():
            dist = euclidean_distances(x.T)
            most_distance_columns_indices = get_top_most_different(dist, top_number)
            columns = x.columns[most_distance_columns_indices]
            fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(25, 5))
            print(f"Top {top_number} different features from {region}.")
            for column, axis in zip(columns, axes.flatten()):
                head, tail = x[column].quantile([0.05, 0.95]).values.ravel()

                mask = ((x[column] < tail) & (x[column] > head)).values

                cleared_x = x[column][mask]
                cleared_y = labels[region].values.ravel()[mask]

                cleared_x[cleared_y==0].hist(ax=axis, bins=20)
                cleared_x[cleared_y==1].hist(ax=axis, bins=20)

                axis.set_title(column)
            fig.tight_layout()
            plt.show()
            
    def most_different_tuples(top_number: int) -> None:
        for region, x in epigenomes.items():
            dist = euclidean_distances(x.T)
            dist = np.triu(dist)
            tuples = get_top_most_different_tuples(dist, top_number)
            fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(25, 5))
            print(f"Top {top_number} different tuples of features from {region}.")
            for (i, j), axis in zip(tuples, axes.flatten()):
                column_i = x.columns[i]
                column_j = x.columns[j]
                for column in (column_i, column_j):
                    head, tail = x[column].quantile([0.05, 0.95]).values.ravel()
                    mask = ((x[column] < tail) & (x[column] > head)).values
                    x[column][mask].hist(ax=axis, bins=20, alpha=0.5)
                axis.set_title(f"{column_i} and {column_j}")
            fig.tight_layout()
            plt.show()
        
    correlated(3)
    uncorrelated(3)
    most_different_features(5)
    most_different_tuples(5)

In [ ]:
# Data analysis (selection and decomposition)

In [ ]:
def get_filtered_with_boruta(epigenomes: Dict[str, pd.DataFrame], labels: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
    def get_features_filter(data: pd.DataFrame, label: pd.DataFrame, name: str) -> BorutaPy:
        boruta_selector = BorutaPy(
            RandomForestClassifier(n_jobs=cpu_count(), class_weight='balanced', max_depth=5),
            n_estimators='auto',
            verbose=2,
            alpha=0.05, # p_value
            max_iter=10, # In practice one would run at least 100-200 times
            random_state=42
        )
        boruta_selector.fit(data.values, label.values.ravel())
        return boruta_selector

    return {
        region: get_features_filter(
            data=data,
            label=labels[region],
            name=f"{cell_line}/{region}"
        ).transform(data.values)
        for region, data in tqdm(
            epigenomes.items(),
            desc="Running Baruta Feature estimation"
        )
    }

In [ ]:
def get_tasks(epigenomes: Dict[str, pd.DataFrame]):
    def check_tasks(xs, ys, titles: List[str]) -> Tuple:
        assert len(xs) == len(ys) == len(titles)
        for x, y in zip(xs, ys):
            assert x.shape[0] == y.shape[0]
        return xs, ys, titles
    
    tasks = {
        "x":[
            *[
                val.values
                for val in epigenomes.values()
            ],
            *[
                val.values
                for val in sequences.values()
            ],
            pd.concat(sequences.values()).values,
            pd.concat(sequences.values()).values,
            *[
                np.hstack([
                    pca(epigenomes[region], n_components=25),
                    mfa(sequences[region], n_components=25)
                ])
                for region in epigenomes
            ]
        ],
        "y":[
            *[
                val.values.ravel()
                for val in labels.values()
            ],
            *[
                val.values.ravel()
                for val in labels.values()
            ],
            pd.concat(labels.values()).values.ravel(),
            np.vstack([np.ones_like(labels["promoters"]), np.zeros_like(labels["enhancers"])]).ravel(),
            *[
                val.values.ravel()
                for val in labels.values()
            ],
        ],
        "titles":[
            "Epigenomes promoters",
            "Epigenomes enhancers",
            "Sequences promoters",
            "Sequences enhancers",
            "Sequences active regions",
            "Sequences regions types",
            "Combined promoters data",
            "Combined enhancers data"
        ]
    }
    
    return check_tasks(tasks['x'], tasks['y'], tasks['title'])

In [ ]:
def get_decomposed_data(xs, ys, titles):
    colors = np.array([
        "tab:blue",
        "tab:orange",
    ])
    
    def pca(data: np.ndarray, n_components: int = 2) -> np.ndarray:
        return PCA(n_components=n_components, random_state=42).fit_transform(data)

    def mfa(data: pd.DataFrame, n_components : int = 2, nucleotides : str = 'actg') -> np.ndarray:
        return MFA(groups={
            nucleotide: [
                column
                for column in data.columns
                if nucleotide in column
            ]
            for nucleotide in nucleotides
        }, n_components=n_components, random_state=42).fit_transform(data)
    
    def sklearn_tsne(x: np.ndarray, perplexity: int, dimensionality_threshold: int = 50):
        if x.shape[1] > dimensionality_threshold:
            x = pca(x, n_components=dimensionality_threshold)
        return STSNE(perplexity=perplexity, n_jobs=cpu_count(), random_state=42).fit_transform(x)
    
    def ulyanov_tsne(x: np.ndarray, perplexity: int, dimensionality_threshold: int = 50, n_components: int = 2):
        if x.shape[1] > dimensionality_threshold:
            x = pca(x, n_components=dimensionality_threshold)
        return UTSNE(n_components=n_components, perplexity=perplexity, n_jobs=cpu_count(), random_state=42, verbose=True).fit_transform(x)
    
    def cannylab_tsne(x: np.ndarray, perplexity: int, dimensionality_threshold: int = 50):
        if x.shape[1] > dimensionality_threshold:
            x = pca(x, n_components=dimensionality_threshold)
        return CTSNE(perplexity=perplexity, random_seed=42).fit_transform(x)
    
    def nystroem(x:np.array)->np.array:
        return Nystroem(random_state=42, n_components=300).fit_transform(x)

    def monte_carlo(x:np.array)->np.array:
        return RBFSampler(random_state=42, n_components=300).fit_transform(x)

    def linear(x:np.array)->np.array:
        return x
    
    

In [ ]:
def show_decomposed_data():
    def show_pca():
        fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(32, 16))
        for x, y, title, axis in tqdm(zip(xs, ys, titles, axes.flatten()), desc="Computing PCAs", total=len(xs)):
            axis.scatter(*pca(x).T, s=1, color=colors[y])
            axis.xaxis.set_visible(False)
            axis.yaxis.set_visible(False)
            axis.set_title(f"PCA decomposition - {title}")
        plt.show()
        
    def show_tsne():
        for perpexity in tqdm((30, 40, 50, 100, 500, 5000), desc="Running perplexities"):
            fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(40, 20))
            for x, y, title, axis in tqdm(zip(xs, ys, titles, axes.flatten()), desc="Computing TSNEs", total=len(xs)):
                axis.scatter(*cannylab_tsne(x, perplexity=perpexity).T, s=1, color=colors[y])
                axis.xaxis.set_visible(False)
                axis.yaxis.set_visible(False)
                axis.set_title(f"TSNE decomposition - {title}")
            fig.tight_layout()
            plt.show()